# Word Embeddings : le modèle Word2Vec

## Imports

In [3]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [4]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [5]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [6]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [7]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [8]:
len(bigram_phrases.vocab.keys())

4574835

Prenons une clé au hasard :

In [9]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

taxes_communale


Le dictionnaire indique le score de cette coocurrence :

In [10]:
bigram_phrases.vocab[key_]

6

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [12]:
%time bigram_phrases[sentences[78]]

CPU times: user 250 µs, sys: 0 ns, total: 250 µs
Wall time: 256 µs


['maladie',
 ',',
 'le',
 'conseil',
 'ajourne',
 'l',
 "'",
 'examen',
 'de',
 'la',
 'reclamation',
 'des',
 'tailleurs',
 'de',
 'pierre',
 'et',
 'marchands',
 'de',
 'briques',
 ',',
 'contre',
 'quelques',
 'articles',
 'des',
 'taxes_communales',
 ',',
 'et',
 'la',
 'discussion',
 'des',
 'budgets',
 'de',
 'la']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [13]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [14]:
%time bigram_phraser[sentences[78]]

CPU times: user 137 µs, sys: 2 µs, total: 139 µs
Wall time: 153 µs


['maladie',
 ',',
 'le',
 'conseil',
 'ajourne',
 'l',
 "'",
 'examen',
 'de',
 'la',
 'reclamation',
 'des',
 'tailleurs',
 'de',
 'pierre',
 'et',
 'marchands',
 'de',
 'briques',
 ',',
 'contre',
 'quelques',
 'articles',
 'des',
 'taxes_communales',
 ',',
 'et',
 'la',
 'discussion',
 'des',
 'budgets',
 'de',
 'la']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [15]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [16]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [17]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [18]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de'], ['bruxelles'], ['bulletin', 'ires', '8eanas'], ['dl', '!'], ['conseil'], ['communal'], ['annee'], ['1847', '.'], ['au'], ['ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin'], ['conseil'], ['aes'], ['seances'], ['communal', '.'], ['annee'], ['1847', '.'], ['bruxelles', ','], ['imprimerie'], ['d', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ','], ['rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ','], ['1', '84', '8'], ['de', '!'], ['du', 'consei'], ['dibi'], ['e', '.', '-', 'communication'], ['conclusions', 'de', 'la', 'section', 'des'], ['du', 'nouvel', 'hospice', 'pour', 'les', 'av'], ['enraisonde', 'l', "'", 'absence', '&'], ['maladie', '.', 'le', 'conseil', 'ajourne'], ['leurs', 'de', 'pierre', 'el', 'marchai'], ['cles', 'des', 'taxes', 'communale', "'"], ['bieniaance', 'eldeseianv'], ['il', 'est', 'donne', 'communie', ';'], ['mandant', 'le', 'o', 'p', 'fa'], ['gn

## Entrainement d'un modèle Word2Vec sur ce corpus

In [19]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 9min 13s, sys: 7.85 s, total: 9min 21s
Wall time: 9min 24s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [20]:
outfile = f"../data/bulletins.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [21]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [22]:
model.wv["bruxelles"]

array([ 2.704708  ,  2.085041  ,  3.3235934 , -1.8787369 , -0.02741746,
       -0.12021404, -1.2234093 ,  2.9869623 , -0.17068432, -2.128259  ,
        0.99761   ,  2.119209  , -1.1309747 ,  1.9073141 , -0.5888142 ,
        1.3720771 ,  0.20062311,  0.62642306,  3.397639  , -1.0681671 ,
       -0.46656176, -1.968571  , -1.1874839 , -2.4281697 , -2.4578912 ,
       -2.7730339 , -1.6315868 ,  4.071529  ,  1.4264663 , -3.2292264 ,
       -0.13025007, -5.2294874 ], dtype=float32)

### Calculer la similarité entre deux termes

In [47]:
model.wv.similarity("boucher", "boulanger")

0.89166415

### Chercher les mots les plus proches d'un terme donné

In [28]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8189939856529236),
 ('liege', 0.8033068180084229),
 ('lille', 0.7978021502494812),
 ('nivelles', 0.7936449646949768),
 ('paris', 0.792253315448761),
 ('gand', 0.7523193955421448),
 ('tournai', 0.7189964652061462),
 ('rruxelles', 0.7156846523284912),
 ('louvain', 0.709695041179657),
 ('bruges', 0.6986990571022034)]

### Faire des recherches complexes à travers l'espace vectoriel

In [46]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('gand', 0.9031482338905334), ('liege', 0.8951103091239929), ('paris', 0.8824741840362549), ('lille', 0.8808256387710571), ('londres', 0.8793736100196838), ('verviers', 0.8520703911781311), ('berlin', 0.8428133130073547), ('tournai', 0.8380563259124756), ('hambourg', 0.8355967998504639), ('ostende', 0.8263471126556396)]
